In [1]:
%%time 

import tensorrt as trt
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
import cv2
import torch 
f = open("/home/pat/scratch_dtu_synced/yolor/runs/train/yolor_csp5/weights/best_ap.trt", "rb")
# f = open("/home/pat/scratch/yolor/alexnet.trt", "rb")
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING)) 

engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

/home/pat/miniconda3/envs/yolor/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[06/23/2022-12:57:44] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0
[06/23/2022-12:57:44] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0
CPU times: user 952 ms, sys: 595 ms, total: 1.55 s
Wall time: 3.57 s


In [3]:
# need to set input and output precisions to FP16 to fully enable it
BATCH_SIZE=1
target_dtype=np.float32
img = cv2.imread("/home/pat/scratch/RumexWeeds/20210806_hegnstrup/seq11/imgs/20210806_hegnstrup_rgb_11_1628251306926751800.png")
img = cv2.resize(img, (640,640))
img = np.expand_dims(img, 0)
img = img.astype(np.float32)
img /= 255
input_batch = img
input_batch = input_batch.swapaxes(-1, 1)
print(input_batch.shape)
print(input_batch.dtype)


(1, 3, 640, 640)
float32


In [5]:
np.empty(1, dtype=np.float32).nbytes

4

In [6]:
output = np.empty([BATCH_SIZE, 25200, 6], dtype=np.float32)
output2 = np.empty([BATCH_SIZE, 3, 80, 80, 6], dtype=np.float32)
output3 = np.empty([BATCH_SIZE, 3, 40, 40, 6], dtype=np.float32)
output4 = np.empty([BATCH_SIZE, 3, 20, 20, 6], dtype=np.float32)


In [15]:
output_cuda = np.empty((output.nbytes + output2.nbytes + output3.nbytes + output4.nbytes)//4, dtype=np.float32)


In [16]:


# output = np.empty([1, 1000], dtype=np.float32)

# allocate device memory
d_input = cuda.mem_alloc(1 * input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output_cuda.nbytes)


# d_output = cuda.mem_alloc(1* output.nbytes)

bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()

In [17]:
def predict(batch): # result gets copied into output
    # transfer input data to device
    print("Copying batch to cuda device")
    cuda.memcpy_htod_async(d_input, batch, stream)
    # execute model
    print("Executing inference")
    context.execute_async_v2(bindings, stream.handle, None)
    # transfer predictions back
    print("transfering back")
    cuda.memcpy_dtoh_async(output_cuda, d_output, stream)
    # syncronize threads
    print("synchronizing")
    stream.synchronize()
    
    return output_cuda

In [18]:
print("Warming up...")

pred = predict(input_batch)

print("Done warming up!")

Warming up...
Copying batch to cuda device
Executing inference
transfering back
[06/23/2022-13:01:33] [TRT] [E] 3: [executionContext.cpp::enqueueInternal::342] Error Code 3: API Usage Error (Parameter check failed at: runtime/api/executionContext.cpp::enqueueInternal::342, condition: (mEngine.bindingIsInput(x) && hasZeroVolume(dims)) || bindings[x]
)
synchronizing
Done warming up!


In [20]:
predict(input_batch)

Copying batch to cuda device
Executing inference
transfering back
[06/23/2022-13:01:42] [TRT] [E] 1: [runner.cpp::execute::418] Error Code 1: Myelin (Final synchronize failed (700))


LogicError: cuMemcpyDtoHAsync failed: an illegal memory access was encountered